# 1번 레이어 : 합성곱층(Convolutional layer)
합성곱(in_channel = 1, out_channel = 32, kernel_size= 3, stride = 1, padding = 1) + 활성화 함수 ReLU
맥스플링(kernel_size= 2, stride=2)

# 2번 레이어 : 합성곱층(Convolutional layer)
합성곱(in_channel = 32, out_channel = 64, kernel_size= 3, stride = 1, padding = 1) + 활성화 함수 ReLU
맥스플링(kernel_size= 2, stride=2)

# 3번 레이어 : 전결합층(Fully-Connected layer)
특성맵을 펼친다. # batch_size x 7 x 7 x 64 -> batch_size x 3136
전결합층(뉴런 10개) + 활성화 함수 Softmax

In [1]:
import torch
import torch.nn as nn

In [9]:
inputs = torch.Tensor(1, 1, 28, 28)
print(inputs.size())

torch.Size([1, 1, 28, 28])


In [4]:
conv1=nn.Conv2d(1, 32, 3, padding=1)
print(conv1)

Conv2d(1, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [5]:
conv2=nn.Conv2d(32, 64, kernel_size=3, padding=1)
print(conv2)

Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))


In [6]:
pool=nn.MaxPool2d(2)
print(pool)

MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)


In [8]:
out=conv1(inputs)
print(out.size())

torch.Size([1, 32, 28, 28])


In [10]:
out=pool(out)
print(out.size())

torch.Size([1, 32, 14, 14])


In [11]:
out=conv2(out)
print(out.size())

torch.Size([1, 64, 14, 14])


In [12]:
out=pool(out)
print(out.size())

torch.Size([1, 64, 7, 7])


In [13]:
# 차원이 아주 크기 때문에 차원을 고쳐보자
out=out.view(out.size(0),-1)
print(out.size())

torch.Size([1, 3136])


In [14]:
fc=nn.Linear(3136, 10)
out=fc(out)
print(out.size())

torch.Size([1, 10])


In [16]:
import torch
import torchvision.datasets as dsets
import torchvision.transforms as transforms
import torch.nn.init

In [17]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
#랜덤 시드 고정
torch.manual_seed(777)
#GPU 사용 가능일 경우 랜덤 시드 고정
if device == 'cuda':
    torch.cuda.manual_seed_all(777)

In [18]:
learning_rate = 0.001
training_epochs = 30
batch_size = 100

In [19]:
mnist_train=dsets.MNIST(root="MNIST_data/", train=True, transform=transforms.ToTensor(), # 다운로드 경로 지정
                        download=True) # True를 지정하념 훈련 데이터로 다운로드 , 텐서로 변환
mnist_test=dsets.MNIST(root="MNIST_data/", train=False, transform=transforms.ToTensor(), 
                        download=True)

In [21]:
from torch.utils.data import DataLoader
data_loader=DataLoader(dataset=mnist_train,batch_size=batch_size,shuffle=True, drop_last=True)

In [23]:
class CNN(torch.nn.Module):
    def __init__(self):
        super().__init__()
        # 1 layer image : (?, 28, 28, 1) , Conv : (?, 28, 28, 32), pool : (?, 14, 14. 32)
        self.layer1=nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        # 2 layer image : (?, 14, 14, 32) , Conv : (?, 14, 14, 64), pool : (?, 7, 7. 64)
        self.layer2=nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=3, stride=1, padding=1),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )

        self.fc=nn.Linear(7*7*64, 10 , bias=True) # 완전 연결층 7*7*64 -> 10

        nn.init.xavier_uniform_(self.fc.weight) # 완전연결층 한정 가중치 초기화

    def forward(self, x):
        out=self.layer1(x)
        out=self.layer2(out)
        out=out.view(out.size(0), -1)
        out=self.fc(out)
        return(out)

In [24]:
model=CNN().to(device)
criterinon=nn.CrossEntropyLoss().to(device)
optimizer=torch.optim.Adam(model.parameters(), lr=learning_rate)

In [25]:
total_batch=len(data_loader)
print("총 배치 수:{}".format(total_batch))

총 배치 수:600


In [26]:
for epoch in range(training_epochs):
    avg_cost=0

    for X, Y in data_loader:
        X=X.to(device)
        Y=Y.to(device)

        optimizer.zero_grad()
        y_hat=model(X)
        cost=criterinon(y_hat, Y)
        cost.backward()
        optimizer.step()

        avg_cost +=cost/total_batch
    print("Epoch:{:>4} cost={:>.9f}".format(epoch+1, avg_cost))    

Epoch:   1 cost=0.225634485
Epoch:   2 cost=0.063048124
Epoch:   3 cost=0.046261337
Epoch:   4 cost=0.037375215
Epoch:   5 cost=0.031302862
Epoch:   6 cost=0.026022289
Epoch:   7 cost=0.021658676
Epoch:   8 cost=0.018033143
Epoch:   9 cost=0.016198572
Epoch:  10 cost=0.013169923
Epoch:  11 cost=0.009722571
Epoch:  12 cost=0.009766354
Epoch:  13 cost=0.008609849
Epoch:  14 cost=0.006950282
Epoch:  15 cost=0.007148594
Epoch:  16 cost=0.005402076
Epoch:  17 cost=0.004118839
Epoch:  18 cost=0.004830286
Epoch:  19 cost=0.005046954
Epoch:  20 cost=0.003640720
Epoch:  21 cost=0.002884196
Epoch:  22 cost=0.003938336
Epoch:  23 cost=0.004305478
Epoch:  24 cost=0.003056651
Epoch:  25 cost=0.000949860
Epoch:  26 cost=0.002693200
Epoch:  27 cost=0.004433095
Epoch:  28 cost=0.001017497
Epoch:  29 cost=0.002444602
Epoch:  30 cost=0.002188499


In [27]:
with torch.no_grad():
  X_test = mnist_test.test_data.view(len(mnist_test), 1, 28, 28).float().to(device)
  Y_test = mnist_test.test_labels.to(device)

  prediction = model(X_test)
  correct_prediction = torch.argmax(prediction, 1) == Y_test
  accuracy = correct_prediction.float().mean()
  print('Accuracy:', accuracy.item())

c:\Users\admin\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:80: UserWarning: test_data has been renamed data
  warnings.warn("test_data has been renamed data")
c:\Users\admin\anaconda3\lib\site-packages\torchvision\datasets\mnist.py:70: UserWarning: test_labels has been renamed targets
  warnings.warn("test_labels has been renamed targets")


Accuracy: 0.9857000112533569


In [ ]:
mnist_train=dsets.CIFAR10(root="cifar10_data/", train=True, transform=transforms.ToTensor(), # 다운로드 경로 지정
                        download=True) # True를 지정하념 훈련 데이터로 다운로드 , 텐서로 변환
mnist_test=dsets.CIFAR10(root="cifar10_data/", train=False, transform=transforms.ToTensor(), 
                        download=True)